# --------------------Pre_training--------------------

# Import packages

In [10]:
#python 3.6.13
import numpy as np
import pandas as pd
import os
import keras
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler

In [11]:
#num_classes
p=2
#shape
shp=140
#wtraining data & testing data link
link="C:/Users/ASUS/Desktop/cg working/HRV/Identify correct wave methods/Deep-Neural-Network-For-Heartbeat-Classification-master/models/"

#ita for focal loss function
ita=0

# parameters in paper ita=2, epoch=50, batch size=512, CNN_kernel_size=(11,5,3), CNN_kernel_num=(16,32,64), CNN_strides=(3,1,1), 
# max_pool=(3,2), final_relu=64
# parameters of mine ita=, epoch=50, batch size=512, CNN_kernel_size=(6,11,4), CNN_kernel_num=(32,32,64), CNN_strides=(2,1,1), 
# max_pool=(3,2), final_relu=16

# pre_training function

In [12]:
def f1(y_true, y_pred):
    y_pred = K.one_hot(K.argmax(y_pred, axis=-1), num_classes=p)
    y_true = K.cast(y_true, y_pred.dtype)

    tp = K.sum(y_true * y_pred, axis=0)
    tn = K.sum((1 - y_true) * (1 - y_pred), axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    return K.mean(2 * precision * recall / (precision + recall + K.epsilon()))


def categorical_focal_loss(gamma=ita):
    """
        Categorical form of focal loss.
            FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
        References:
            https://arxiv.org/pdf/1708.02002.pdf
        Usage:
            model.compile(loss=categorical_focal_loss(gamma=2), optimizer="adam", metrics=["accuracy"])
            model.fit(class_weight={0:alpha0, 1:alpha1, ...}, ...)
        Notes:
           1. The alpha variable is the class_weight of keras.fit, so in implementation of the focal loss function
           we needn't define this variable.
           2. (important!!!) The output of the loss is the loss value of each training sample, not the total or average
            loss of each batch.
    """

    def focal_loss(y_true, y_pred):
        y_true = K.cast(y_true, y_pred.dtype)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        return K.sum(K.sum(-y_true * K.pow(1 - y_pred, gamma) * K.log(y_pred), axis=-1))

    return focal_loss


def load_data(filename):
    #load pkl file
    import pickle

    with open(filename, "rb") as f:
        (x1_train, x2_train, y_train), (x1_test, x2_test, y_test) = pickle.load(f)

    return (x1_train, x2_train, y_train), (x1_test, x2_test, y_test)


def pre_training(filename):
    batch_size = 512

    # loading data
    (x1_train, x2_train, y_train), (x1_test, x2_test, y_test) = load_data(filename)

    x1_train = np.expand_dims(x1_train, axis=-1)
    x1_test = np.expand_dims(x1_test, axis=-1)

    scaler = RobustScaler()
    x2_train = scaler.fit_transform(x2_train)
    x2_test = scaler.transform(x2_test)

    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes=p)
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes=p)

    model = load_model(os.path.join(link, "model_crossentropy.h5"),
                       custom_objects={"focal_loss": categorical_focal_loss(gamma=ita),
                                       "f1": f1})
    model.summary()

    #result(train)
    print("training:")
    y_true = np.argmax(y_train, axis=-1)
    y_pred = np.argmax(model.predict([x1_train, x2_train], batch_size=batch_size, verbose=1), axis=-1)

    print("row:true, column:pred")
    print(pd.DataFrame(confusion_matrix(y_true, y_pred)))
    print("classification_report")
    print(classification_report(y_true, y_pred, digits=4))

    #result(test)
    print("testing:")
    y_true = np.argmax(y_test, axis=-1)
    y_pred = np.argmax(model.predict([x1_test, x2_test], batch_size=batch_size, verbose=1), axis=-1)

    print("row:true, column:pred")
    print(pd.DataFrame(confusion_matrix(y_true, y_pred)))
    print("classification_report")
    print(classification_report(y_true, y_pred, digits=4))



# predict new wave (marked data)

In [13]:
link2="C:/Users/ASUS/Desktop/cg working/HRV/Identify correct wave methods/Deep-Neural-Network-For-Heartbeat-Classification-master/"
CNNMPFL_modeling=pre_training(os.path.join(link2, "CNNMPFL_preprocess.pkl"))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 140, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 67, 16)       144         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 67, 16)       64          conv1d_3[0][0]                   
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 67, 16)       0           batch_normalization_3[0][0]      
____________________________________________________________________________________________

# predict new wave (unmarked data)

In [42]:
def load_data(filename):
    #load pkl file
    import pickle

    with open(filename, "rb") as f:
        (x1_train, x2_train), (x1_test, x2_test) = pickle.load(f)

    return (x1_train, x2_train), (x1_test, x2_test)

batch_size = 512
# loading data
(x1_train, x2_train), (x1_test, x2_test) = load_data(os.path.join(link2, "CNNMPFL_newwave_preprocess.pkl"))

x1_train = np.expand_dims(x1_train, axis=-1)
x1_test = np.expand_dims(x1_test, axis=-1)

scaler = RobustScaler()
x2_train = scaler.fit_transform(x2_train)
x2_test = scaler.transform(x2_test)

model = load_model(os.path.join(link, "model_crossentropy.h5"),
                       custom_objects={"focal_loss": categorical_focal_loss(gamma=ita),
                                       "f1": f1})
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 140, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 67, 16)       144         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 67, 16)       64          conv1d_3[0][0]                   
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 67, 16)       0           batch_normalization_3[0][0]      
____________________________________________________________________________________________

In [43]:
#result(train)
print("training:")
y_pred = np.argmax(model.predict([x1_train, x2_train], batch_size=batch_size, verbose=1), axis=-1)
y_pred

training:
3/3 [==============================] - 0s 6ms/step


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [44]:
table={"classfication":["count","prop."],
       "Normal":[len(y_pred)-sum(y_pred),1-sum(y_pred)/len(y_pred)],
       "Abnormal":[sum(y_pred),sum(y_pred)/len(y_pred)]}
table=pd.DataFrame(table)
table

,classfication,Normal,Abnormal
0,count,1141.000000,91.000000
1,prop.,0.926136,0.073864


In [45]:
#result(test)
print("testing:")
y_pred = np.argmax(model.predict([x1_test, x2_test], batch_size=batch_size, verbose=1), axis=-1)
y_pred

testing:
1/1 [==============================] - 0s 15ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [46]:
table={"classfication":["count","prop."],
       "Normal":[len(y_pred)-sum(y_pred),1-sum(y_pred)/len(y_pred)],
       "Abnormal":[sum(y_pred),sum(y_pred)/len(y_pred)]}
table=pd.DataFrame(table)
table

,classfication,Normal,Abnormal
0,count,283.000000,25.000000
1,prop.,0.918831,0.081169
